In [1]:
import dash
from dash import html
import dash_cytoscape as cyto
import csv
import networkx as nx
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

# Load data from CSV and build a NetworkX graph
G_nx = nx.Graph()
with open('mapper_graph_adjacency_list.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip the header if present
    for row in reader:
        G_nx.add_edge(row[0], row[1])

elements = [
    {'data': {'id': node, 'label': node}} for node in G_nx.nodes()
] + [
    {'data': {'source': edge[0], 'target': edge[1]}} for edge in G_nx.edges()
]

stylesheet = [
    {
        'selector': 'node',
        'style': {
            'background-color': 'gray',
            'label': 'data(label)',
            'font-size': '1px',
            'width': '3px',
            'height': '3px'
        }
    },
    {
        'selector': 'node:selected',
        'style': {
            'background-color': 'red'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': 'light-gray',
            'width': 0.1
        }
    }
]

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-graph',
        elements=elements,
        stylesheet=stylesheet,
        layout={'name': 'cose'},
        style={'width': '100%', 'height': '900px'},
        boxSelectionEnabled=True
    ),
    html.Pre(id='selected-node-data', style={'padding-top': '20px'})
])

@app.callback(
    Output('selected-node-data', 'children'),
    [Input('cytoscape-graph', 'selectedNodeData')]
)
def display_selected_node_data(data):
    if not data:
        return "No nodes selected"
    else:
        selected_info = '\n'.join([f"{node['id']}" for node in data])
        return f"Selected Nodes:\n{selected_info}"

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)


In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

nodes_data_path = 'nodes_to_rows.csv'
nodes_data = pd.read_csv(nodes_data_path)

dataset_path = 'df_mular_median (1).csv'
original_dataset = pd.read_csv(dataset_path)

def show_rows(input_text):
    try:
        # Parsing the input from the TextArea widget
        node_ids = input_text.split('\n')  # Splits the input by newlines
        all_rows_list = []
        for node_id in node_ids:
            node_id = node_id.strip()  # Removes any leading/trailing whitespace
            if node_id:  # Checks if node_id is not empty
                rows_str = nodes_data.loc[nodes_data['node_id'] == node_id, 'row_indices'].values[0]
                rows_list = list(map(int, rows_str.split(',')))
                all_rows_list.extend(rows_list)
        
        unique_rows_list = list(set(all_rows_list))
        selected_rows = original_dataset.iloc[unique_rows_list]
        clear_output(wait=True)
        display(selected_rows)
    except Exception as e:
        clear_output(wait=True)
        print(f"An error occurred: {e}")

# TextArea widget for node IDs input
node_id_input = widgets.Textarea(
    value='',
    placeholder='Type or paste node IDs here, each on a new line',
    description='Node IDs:',
    disabled=False,
    layout=widgets.Layout(width='300px', height='100px')
)

# Button to trigger data display
button = widgets.Button(description="Show Data")

# Output widget to display selected data or errors
output = widgets.Output()

def on_button_clicked(b):
    with output:
        show_rows(node_id_input.value)

button.on_click(on_button_clicked)

# Displaying the widgets
display(node_id_input, button, output)


Textarea(value='', description='Node IDs:', layout=Layout(height='100px', width='300px'), placeholder='Type or…

Button(description='Show Data', style=ButtonStyle())

Output()